In [2]:
import os
os.getcwd()

'/Users/tanvirahmed/Desktop/delivery-orderflow-ml/notebooks/item_sales'

### 1. Load data + parse dates

In [6]:
import pandas as pd

df = pd.read_csv("../../data/item_sales/grocery_sales_autumn_2025.csv")
df["date"] = pd.to_datetime(df["date"])
df.head()

,date,venue_id,sku_id,phl1_id,phl2_id,phl3_id,operating_minutes,country_id,price,promo_flag,promo_depth,in_stock_minutes,stockout_flag,units_sold
0,2025-09-01,V001,S0001,P1_01,P2_03,P3_009,900,FI,0.55,0,0.0,900,0,67
1,2025-09-01,V001,S0002,P1_01,P2_03,P3_009,900,FI,0.55,0,0.0,900,0,45
2,2025-09-01,V001,S0003,P1_01,P2_03,P3_009,900,FI,0.65,0,0.0,900,0,32
3,2025-09-01,V001,S0004,P1_07,P2_21,P3_063,900,FI,1.75,0,0.0,900,0,51
4,2025-09-01,V001,S0005,P1_07,P2_21,P3_063,900,FI,1.15,0,0.0,900,0,39


In [7]:
df.dtypes

date                 datetime64[us]
venue_id                        str
sku_id                          str
phl1_id                         str
phl2_id                         str
phl3_id                         str
operating_minutes             int64
country_id                      str
price                       float64
promo_flag                    int64
promo_depth                 float64
in_stock_minutes              int64
stockout_flag                 int64
units_sold                    int64
dtype: object

### 2. Sort by SKU and date

#### Sorts each SKU’s history in time order. Lag features must follow the correct time sequence.

In [8]:
df = df.sort_values(["sku_id", "date"]).reset_index(drop=True)

### 3. Create time-series lag features

#### Yesterday’s sales (lag 1)

In [9]:
df["lag_1"] = df.groupby("sku_id")["units_sold"].shift(1)

#### Sales from last week (lag 7)

In [12]:
df["lag_7"] = df.groupby("sku_id")["units_sold"].shift(7)

### 4. Rolling averages (trend features)

#### Rolling average of last 7 days

In [18]:
df["rolling_7"] = (
    df.groupby("sku_id")["units_sold"]
      .shift(1)
      .rolling(7)
      .mean()
)

#### Rolling average of last 14 days

In [19]:
df["rolling_14"] = (
    df.groupby("sku_id")["units_sold"]
      .shift(1)
      .rolling(14)
      .mean()
)

### 5. Calendar features (weekday & weekend)

In [20]:
df["weekday"] = df["date"].dt.weekday
df["weekend"] = df["weekday"].isin([5, 6]).astype(int)

### 6. Stock availability feature

In [23]:
df["stock_ratio"] = df["in_stock_minutes"] / df["operating_minutes"]

###  7. Drop rows where lag/rolling are missing
#### After creating lag features, the first few days for each SKU will have NaN.

In [25]:
df_fe = df.dropna(subset=["lag_1", "lag_7", "rolling_7", "rolling_14"]).reset_index(drop=True)
df_fe.shape

(37660, 21)

### 8. Select features and target

In [30]:
features = [
    "lag_1",
    "lag_7",
    "rolling_7",
    "rolling_14",
    "promo_flag",
    "promo_depth",
    "stock_ratio",
    "weekday",
    "weekend",
    "price"
]

X = df_fe[features]
y = df_fe["units_sold"]

### 9. Save processed dataset

In [32]:
df_fe.to_csv("../../data/item_sales/processed_sales_features.csv", index=False)